<a href="https://colab.research.google.com/github/mint-aguccim/Machine_Learning_Programming/blob/main/01_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필수 라이브러리 설치
!pip install finance-datareader mplfinance yfinance gnews

# 2. 구글 드라이브 연결 및 폴더 구조 생성
import os
from google.colab import drive

drive.mount('/content/drive')

# 프로젝트 기본 경로
base_path = '/content/drive/MyDrive/P실무프로젝트_3조'

# 필요한 폴더들 (데이터, 주식이미지, 코인이미지, 모델저장소)
required_folders = ['data', 'images', 'bitcoin_images', 'models']

print("폴더 구조 확인 및 생성 중...")
for folder in required_folders:
    folder_path = os.path.join(base_path, folder)
    os.makedirs(folder_path, exist_ok=True)
    print(f" - 확인 완료: {folder_path}")

print("\n환경 설정이 완료되었습니다!")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 12.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=d9d8dec89f54b569edbed8b976395f07ccc0e7b7f9ad4c6a1a790b8fe4510d93
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
Mounted at /content/drive
폴더 구조 확인 및 생성 중...
 - 확인 완료: /content/drive/MyDrive/P실무프로젝트_3조/data
 - 확인 완료: /content/drive/MyDrive/P실무프로젝트_3조/images
 - 확인 완료: /content/drive/MyDrive/P실무프로젝트_3조/bitcoin_images
 - 확인 완료: /content/drive/MyDrive/P실무프로젝트_3조/models

환경 설정이 완료되었습니다!


In [ ]:
# ==========================================
# [Track A] 삼성전자 차트 이미지 생성
# ==========================================
import FinanceDataReader as fdr
import mplfinance as mpf
import pandas as pd
import matplotlib.pyplot as plt

# 설정
ticker = '005930' # 삼성전자
start_date = '2018-01-01' # 데이터 기간
end_date   = '2024-05-20'
save_path = os.path.join(base_path, 'images')

print(f"📈 [{ticker}] 주가 데이터 다운로드 중...")
df = fdr.DataReader(ticker, start_date, end_date)
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# 스타일 설정
mc = mpf.make_marketcolors(up='r', down='b', inherit=True)
style = mpf.make_mpf_style(marketcolors=mc)
window_size = 20  # window size
count = 0

print(f"🖼️ 이미지 변환 시작 (저장소: {save_path})")
for i in range(len(df) - window_size):
    subset = df.iloc[i : i + window_size]
    target_date = subset.index[-1].strftime('%Y-%m-%d')
    filename = os.path.join(save_path, f"{target_date}.png")

    if not os.path.exists(filename):
        try:
            mpf.plot(subset, type='candle', style=style, axisoff=True, volume=False,
                     savefig=dict(fname=filename, dpi=50, pad_inches=0))
            plt.close('all') # 메모리 정리
            count += 1
        except:
            pass

    if i % 500 == 0: print(f"... {target_date} 처리 중")

print(f"✅ 삼성전자 이미지 생성 완료! (신규 생성: {count}장)")

📈 [005930] 주가 데이터 다운로드 중...
🖼️ 이미지 변환 시작 (저장소: /content/drive/MyDrive/P실무프로젝트_3조/images)
... 2018-01-29 처리 중
... 2020-02-14 처리 중
... 2022-02-22 처리 중
... 2024-03-05 처리 중
✅ 삼성전자 이미지 생성 완료! (신규 생성: 1325장)


In [ ]:
# ==========================================
# [Track B] 비트코인 차트 이미지 생성
# ==========================================
import yfinance as yf
from tqdm.notebook import tqdm

# 설정
ticker = 'BTC-USD'
start_date = '2018-01-01' # 데이터 기간
end_date   = '2024-05-20'
save_path = os.path.join(base_path, 'bitcoin_images')

print(f"🪙 [{ticker}] 데이터 다운로드 중...")
df_coin = yf.download(ticker, start=start_date, end=end_date, progress=False)
if isinstance(df_coin.columns, pd.MultiIndex):
    df_coin.columns = df_coin.columns.get_level_values(0)

# 스타일 설정
mc = mpf.make_marketcolors(up='r', down='b', inherit=True)
style = mpf.make_mpf_style(marketcolors=mc)
window_size = 20 # window_size
count = 0

print(f"🖼️ 이미지 변환 시작 (저장소: {save_path})")
# tqdm으로 진행률 표시
for i in tqdm(range(len(df_coin) - window_size)):
    subset = df_coin.iloc[i : i + window_size]
    target_date = subset.index[-1].strftime('%Y-%m-%d')
    filename = os.path.join(save_path, f"{target_date}.png")

    if not os.path.exists(filename):
        try:
            mpf.plot(subset, type='candle', style=style, axisoff=True, volume=False,
                     savefig=dict(fname=filename, dpi=50, pad_inches=0))
            plt.close('all')
            count += 1
        except:
            pass

print(f"✅ 비트코인 이미지 생성 완료! (신규 생성: {count}장)")

🪙 [BTC-USD] 데이터 다운로드 중...


/tmp/ipython-input-982257350.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_coin = yf.download(ticker, start=start_date, end=end_date, progress=False)


🖼️ 이미지 변환 시작 (저장소: /content/drive/MyDrive/P실무프로젝트_3조/bitcoin_images)


  0%|          | 0/2311 [00:00<?, ?it/s]

✅ 비트코인 이미지 생성 완료! (신규 생성: 2311장)


In [ ]:
# ==========================================
# [News] 뉴스 데이터 크롤링 (투 트랙 시스템)
# ==========================================
import requests
from bs4 import BeautifulSoup
import time
import random
from gnews import GNews

# 데이터 저장 경로
data_folder = os.path.join(base_path, 'data')

# 1. 삼성전자 (네이버) 수집 함수
def collect_naver_stock(keyword, pages=5):
    print(f"🇰🇷 '{keyword}' 네이버 뉴스 수집 중...")
    headers = {"User-Agent": "Mozilla/5.0"}
    news_list = []

    for page in range(1, pages + 1):
        url = f"https://search.naver.com/search.naver?where=news&query={keyword}&start={(page-1)*10+1}&sort=1"
        try:
            res = requests.get(url, headers=headers, timeout=5)
            soup = BeautifulSoup(res.text, "html.parser")
            items = soup.select("div.news_wrap")

            for item in items:
                title = item.select_one("a.news_tit").get_text().strip()
                # 날짜 처리
                date_text = "날짜없음"
                for info in item.select("span.info"):
                    if "전" in info.text or "." in info.text:
                        date_text = info.text.strip()
                        break
                news_list.append([date_text, title])
            time.sleep(random.uniform(0.5, 1.0))
        except: continue

    return pd.DataFrame(news_list, columns=['Date', 'Title'])

# 2. 비트코인 (구글/영어) 수집 함수
def collect_google_coin(keyword):
    print(f"🇺🇸 '{keyword}' 구글(영어) 뉴스 수집 중...")
    google_news = GNews(language='en', country='US', start_date=(2024, 1, 1), end_date=(2024, 5, 20), max_results=1000)
    try:
        json_resp = google_news.get_news(keyword)
        news_list = []
        for item in json_resp:
            title = item.get('title')
            pub_date = item.get('published date')
            # 날짜 변환 시도
            try:
                from dateutil import parser
                date_str = parser.parse(pub_date).strftime('%Y-%m-%d')
            except: date_str = pub_date
            news_list.append([date_str, title])
        return pd.DataFrame(news_list, columns=['Date', 'Title'])
    except: return pd.DataFrame()

# --- 실행 및 저장 ---
# 1. 삼성전자
df_stock = collect_naver_stock("삼성전자", pages=20) # 페이지수 조절 가능
if not df_stock.empty:
    path = os.path.join(data_folder, 'samsung_news_kr.csv')
    df_stock.to_csv(path, index=False, encoding='utf-8-sig')
    print(f"💾 삼성전자 뉴스 저장 완료: {path}")

# 2. 비트코인
df_coin = collect_google_coin("Bitcoin")
if not df_coin.empty:
    path = os.path.join(data_folder, 'bitcoin_news_en.csv')
    df_coin.to_csv(path, index=False, encoding='utf-8-sig')
    print(f"💾 비트코인 뉴스 저장 완료: {path}")

🇰🇷 '삼성전자' 네이버 뉴스 수집 중...
🇺🇸 'Bitcoin' 구글(영어) 뉴스 수집 중...


/tmp/ipython-input-1343173614.py:68: UserWarning: Searches for over 100 articles do not currently support date ranges. 
Start date, end date, and period will be ignored
  df_coin = collect_google_coin("Bitcoin")
/usr/local/lib/python3.12/dist-packages/gnews/gnews.py:237: UserWarning: Only searches using the function get_news support date ranges. Review the documentation for _get_news_more_than_100 for a partial workaround. 
Start date and end date will be ignored
  return self._get_news_more_than_100(key)


💾 비트코인 뉴스 저장 완료: /content/drive/MyDrive/P실무프로젝트_3조/data/bitcoin_news_en.csv
